In [1]:
pip install dotenv google-genai pydantic

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
from google import genai
from pydantic import BaseModel
from google.genai import types
from google.genai.types import Part, GenerateContentConfig
import json


In [ ]:
JSON_MIME_TYPE = "application/json"
PDF_MIME_TYPE = "application/pdf"

os.environ["GEMINI_API_KEY"] = os.get("GEMINI_API_KEY")
client = genai.Client()
model = "gemini-2.0-flash"

In [8]:
class PayslipInfo(BaseModel):
  employee_no: int
  employee_name: str
  gpn: str
  designation: str
  location: str

In [9]:
  with open('Payslip_Oct_2024.pdf', 'rb') as f:
    file_bytes = f.read()

In [10]:
input_prompt = """Analyse the payslips in the form of strings for completeness and correctness.
identify the given fields and return the values in JSON format.
Such as:
1. Employee No
2. Employee Name
3. GPN
4. Designation
5. Location
"""

In [11]:
response = client.models.generate_content(
    model = 'gemini-2.0-flash-001',
    contents = ["The given document is payslip.",
               Part.from_bytes(data=file_bytes, mime_type=PDF_MIME_TYPE)],
    config = GenerateContentConfig(
        system_instruction= input_prompt,
        response_mime_type='application/json',
        response_schema=PayslipInfo,
        temperature=0
    )
)

print(response.text)

{
  "employee_no": 3219904,
  "employee_name": "Akshay Kumar Police Patil",
  "gpn": "IN010137068",
  "designation": "Test Engineer",
  "location": "Bangalore"
}


In [12]:
pdf = pd.DataFrame([json.loads(response.text)])
pdf.to_csv('payslip.csv', index=False)